# Challenger Nestlé 2021

# Planejamento da solução (IOT)

## Input

- Os competidores da batalha de dados da Nestlé terão que apresentar soluções inovadoras, com foco em dados para ajudar a desenvolver o mercado artesanal e independente de produção de bolos e doces.

## Output

- Ferramentas
    - Metabase
    - Streamit
- URL para o time de operação testar
- Download dos dados selecionados

## Tasks

- Descrição dos dados
    - Substituir nulos
- EDA
    - Análise Univariada
    - Análise bivariada
    - Detecção de outliers
    - Seleção de features
    - Estudo do Espaço
    - Definição do Embedding
- Treinamento de algorítimos de ML
    - Análise de performace
    - Análise de clustering
    - Definição dos números de clusters
- EDA Insights
    - Explicação das características dos segmentos
- Tabela para armazenar os clientes clusterizados
- Criação do notebook de produção
- Arquitetura AWS
    - S3 para armazenar dados
    - EC2 para armazenar o ETC e a ferramenta de visualização
    - DB para armazenar a tabela
    - Cronjob
    - Papermil
    

# 0 - Importações e funções

## Importações

In [1]:
import pandas as pd
import numpy  as np
from sklearn    import cluster as c
from sklearn    import metrics as m
from matplotlib import pyplot  as plt



## funções

## Importar dados

In [2]:
df0_raw = pd.read_csv("data/raw/CC_GENERAL.csv", sep=",", low_memory=False)

In [3]:
df0_raw.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


# 1 - Descrição dos dados

In [4]:
df01 = df0_raw.copy()

## Renomear colunas

In [5]:
 # Deixa os nomes das colunas em minúsculo
df01.columns = df01.columns.str.lower()

## dimensão dos dados

In [6]:
print ( 'nro Linhas {}'.format( df01.shape[0]))
print ( 'nro Colunas {}'.format( df01.shape[1]))

nro Linhas 8950
nro Colunas 18


## Tipagens

In [7]:
df01.dtypes

cust_id                              object
balance                             float64
balance_frequency                   float64
purchases                           float64
oneoff_purchases                    float64
installments_purchases              float64
cash_advance                        float64
purchases_frequency                 float64
oneoff_purchases_frequency          float64
purchases_installments_frequency    float64
cash_advance_frequency              float64
cash_advance_trx                      int64
purchases_trx                         int64
credit_limit                        float64
payments                            float64
minimum_payments                    float64
prc_full_payment                    float64
tenure                                int64
dtype: object

## Consultar / Tratar Nulos

In [8]:
df01.isna().sum()

cust_id                               0
balance                               0
balance_frequency                     0
purchases                             0
oneoff_purchases                      0
installments_purchases                0
cash_advance                          0
purchases_frequency                   0
oneoff_purchases_frequency            0
purchases_installments_frequency      0
cash_advance_frequency                0
cash_advance_trx                      0
purchases_trx                         0
credit_limit                          1
payments                              0
minimum_payments                    313
prc_full_payment                      0
tenure                                0
dtype: int64

In [9]:
# percentual de nulos
df01.isna().sum() / df01.shape[0]

cust_id                             0.000000
balance                             0.000000
balance_frequency                   0.000000
purchases                           0.000000
oneoff_purchases                    0.000000
installments_purchases              0.000000
cash_advance                        0.000000
purchases_frequency                 0.000000
oneoff_purchases_frequency          0.000000
purchases_installments_frequency    0.000000
cash_advance_frequency              0.000000
cash_advance_trx                    0.000000
purchases_trx                       0.000000
credit_limit                        0.000112
payments                            0.000000
minimum_payments                    0.034972
prc_full_payment                    0.000000
tenure                              0.000000
dtype: float64

In [10]:
# ver dados nulos
nullMP = df01[df01['minimum_payments'].isnull()]


In [11]:
nullMP.head()

,cust_id,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure
3,C10004,1666.670542,0.636364,1499.0,1499.0,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.0,12
45,C10047,2242.311686,1.000000,437.0,97.0,340.0,184.648692,0.333333,0.083333,0.333333,0.166667,2,5,2400.0,0.000000,NaN,0.0,12
47,C10049,3910.111237,1.000000,0.0,0.0,0.0,1980.873201,0.000000,0.000000,0.000000,0.500000,7,0,4200.0,0.000000,NaN,0.0,12
54,C10056,6.660517,0.636364,310.0,0.0,310.0,0.000000,0.666667,0.000000,0.666667,0.000000,0,8,1000.0,417.016763,NaN,0.0,12
55,C10057,1311.995984,1.000000,1283.9,1283.9,0.0,0.000000,0.250000,0.250000,0.000000,0.000000,0,6,6000.0,0.000000,NaN,0.0,12


In [12]:
prcfull = df01[df01['prc_full_payment'] > 0]

In [13]:
prcfull.head()

,cust_id,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure
1,C10002,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.25,4,0,7000.0,4103.032597,1072.340217,0.222222,12
6,C10007,627.260806,1.000000,7091.01,6402.63,688.38,0.000000,1.000000,1.000000,1.000000,0.00,0,64,13500.0,6354.314328,198.065894,1.000000,12
12,C10013,1516.928620,1.000000,3217.99,2500.23,717.76,0.000000,1.000000,0.250000,0.916667,0.00,0,26,3000.0,608.263689,490.207013,0.250000,12
13,C10014,921.693369,1.000000,2137.93,419.96,1717.97,0.000000,0.750000,0.166667,0.750000,0.00,0,26,7500.0,1655.891435,251.137986,0.083333,12
17,C10018,41.089489,0.454545,519.00,0.00,519.00,0.000000,0.416667,0.000000,0.333333,0.00,0,8,2500.0,254.590662,73.203221,0.250000,12


In [14]:
nullCL = df01[df01['credit_limit'].isnull()]

In [15]:
nullCL.head()

,cust_id,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure
5203,C15349,18.400472,0.166667,0.0,0.0,0.0,186.853063,0.0,0.0,0.0,0.166667,1,0,NaN,9.040017,14.418723,0.0,6


In [16]:
df01['credit_limit'].max()

30000.0

In [17]:
df01.isna().sum()

cust_id                               0
balance                               0
balance_frequency                     0
purchases                             0
oneoff_purchases                      0
installments_purchases                0
cash_advance                          0
purchases_frequency                   0
oneoff_purchases_frequency            0
purchases_installments_frequency      0
cash_advance_frequency                0
cash_advance_trx                      0
purchases_trx                         0
credit_limit                          1
payments                              0
minimum_payments                    313
prc_full_payment                      0
tenure                                0
dtype: int64

# 2 - Filtragem de dados

In [18]:
df02 = df01.copy()

## Separar dados numéricos e categóricos

In [19]:
# Só tem um categórico o ID do usuário que não interessa agora para o dataframe - vou remover aqui
cols_drop = ['cust_id']
df2 = df02.drop(cols_drop, axis=1)


In [20]:
df2.head()

,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure
0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


## Tratar dados Nulos

In [21]:
# Segundo ciclo do CRISP eliminar nulos
# avaliar = minimum_payments colocar zero e credit_limit colocar o valor máximo do dataset
df02['minimum_payments'].fillna((0), inplace=True)
df02['credit_limit'].fillna((df02['credit_limit'].max()), inplace=True)

# 3 - Feature Engineering (Atributos de engenharia)

In [22]:
df03 = df02.copy()

## Avaliar métricas

In [23]:
# separa atributos numéricos
num_att = df03.select_dtypes( include = ['int64','int32', 'float64'] )

In [24]:
# tendencia central - média e mediana
ct1 = pd.DataFrame (num_att.apply( np.mean ) ).T
ct2 = pd.DataFrame (num_att.apply( np.median ) ).T
# dispersão - desvio padrão, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_att.apply (np.std) ).T
d2 = pd.DataFrame(num_att.apply (min ) ).T
d3 = pd.DataFrame(num_att.apply (max ) ).T
d4 = pd.DataFrame(num_att.apply (lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame(num_att.apply (lambda x: x.skew() ) ).T
d6 = pd.DataFrame(num_att.apply (lambda x: x.kurtosis() ) ).T

# concatenar
m = pd.concat( [d2, d3, ct1, ct2, d1, d5, d6, d4] ).T.reset_index()
m.columns = ['Atributos', 'min', 'max','media','Mediana','desv.Padrao','vies','curtose','Range']

In [25]:
m

,Atributos,min,max,media,Mediana,desv.Padrao,vies,curtose,Range
0,balance,0.0,19043.13856,1564.474828,873.385231,2081.415590,2.393386,7.674751,19043.13856
1,balance_frequency,0.0,1.00000,0.877271,1.000000,0.236891,-2.023266,3.092370,1.00000
2,purchases,0.0,49039.57000,1003.204834,361.280000,2136.515413,8.144269,111.388771,49039.57000
3,oneoff_purchases,0.0,40761.25000,592.437371,38.000000,1659.795184,10.045083,164.187572,40761.25000
4,installments_purchases,0.0,22500.00000,411.067645,89.000000,904.287592,7.299120,96.575178,22500.00000
5,cash_advance,0.0,47137.21176,978.871112,0.000000,2097.046713,5.166609,52.899434,47137.21176
6,purchases_frequency,0.0,1.00000,0.490351,0.500000,0.401348,0.060164,-1.638631,1.00000
7,oneoff_purchases_frequency,0.0,1.00000,0.202458,0.083333,0.298319,1.535613,1.161846,1.00000
8,purchases_installments_frequency,0.0,1.00000,0.364437,0.166667,0.397426,0.509201,-1.398632,1.00000
9,cash_advance_frequency,0.0,1.50000,0.135144,0.000000,0.200110,1.828686,3.334734,1.50000


In [26]:
df03.describe()

,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000
mean,1564.474828,0.877271,1003.204834,592.437371,411.067645,978.871112,0.490351,0.202458,0.364437,0.135144,3.248827,14.709832,4497.299233,1733.143852,833.983453,0.153715,11.517318
std,2081.531879,0.236904,2136.634782,1659.887917,904.338115,2097.163877,0.401371,0.298336,0.397448,0.200121,6.824647,24.857649,3648.586787,2895.063757,2335.989918,0.292499,1.338331
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,6.000000
25%,128.281915,0.888889,39.635000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,1.000000,1600.000000,383.276166,163.028164,0.000000,12.000000
50%,873.385231,1.000000,361.280000,38.000000,89.000000,0.000000,0.500000,0.083333,0.166667,0.000000,0.000000,7.000000,3000.000000,856.901546,289.628402,0.000000,12.000000
75%,2054.140036,1.000000,1110.130000,577.405000,468.637500,1113.821139,0.916667,0.300000,0.750000,0.222222,4.000000,17.000000,6500.000000,1901.134317,788.713501,0.142857,12.000000
max,19043.138560,1.000000,49039.570000,40761.250000,22500.000000,47137.211760,1.000000,1.000000,1.000000,1.500000,123.000000,358.000000,30000.000000,50721.483360,76406.207520,1.000000,12.000000


# 4 - EDA - Estudo do espaço

# 5 - Preparação dos dados

# 6 -Seleção de atributos

# 7 - Hiperparametrização

## k-Means

In [27]:
X = df02

In [28]:
clusters = [2,3,4,5,6,7,8]
sil_list = []
dav_list = []
for k in clusters:
    # definição 
    kmeans_model = c.KMeans(n_clusters=k)
    
    # Treino
    kmeans_model.fit(X)
    # predicao
    labels = kmeans_model.predict(X)
    
    # performance
    sil = m.silhouette_score (X, labels)
    dav = m.davies_bouldin_score (X, labels)
    sil_list.append(sil)
    dav_list.append(dav)
    
    

ValueError: could not convert string to float: 'C10001'

In [ ]:
print (sil_list)

In [ ]:
print(dav_list)

In [ ]:
plt.figure(figsize=[20,10])
plt.plot(clusters, sil_list, linestyle = '--', marker = 'o', color = 'b')
plt.xlabel('Nro Clusters');
plt.ylabel('Score da silueta');
plt.title('Score da silueta');

In [ ]:
plt.figure(figsize=[20,10])
plt.plot(clusters, dav_list, linestyle = '--', marker = 'o', color = 'b')
plt.xlabel('Nro Clusters');
plt.ylabel('DB score');
plt.title('DBIndex');

# 8 - Treinamento dos algorítimos

# 9 - Análise de clusters

# 10 - EDA - Descoberta de relatórios - Insights

# 11 - Deploy para produção